In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import adaptive_latents
from adaptive_latents import proSVD, CenteringTransformer, Pipeline
from datasets import Naumann24uDataset

rng = np.random.default_rng()


In [ ]:
d = Naumann24uDataset(Naumann24uDataset.sub_datasets[2])

In [ ]:
in_data = d.C.T

diff_over_time ={
    'over_whole_matrix'       : lambda Qs: (np.linalg.norm(np.diff(Qs, axis=0),axis=(1,2))), 
    'along_axis': lambda Qs: (np.linalg.norm(np.diff(Qs, axis=0),axis=1))
}.get('along_axis')

# prosvd_with_Q_args = dict(output_d=4, init_size=10, pro_arguments=dict(centering=False))

fig, ax = plt.subplots(); ax: plt.Axes

pro = proSVD(k=4, log_level=1)
p = Pipeline([
    CenteringTransformer(),
    pro,
])

p.offline_run_on(in_data)


plt.plot(pro.log['t'][1:], diff_over_time(pro.log['Q']), label='original data')


for time in d.stimulations['sample']:
    plt.axvline(time, color='k')
plt.xlim([1800, 3200])
plt.ylim([0, 0.01])

plt.title(f"Change in Q over time, {pro.Q.shape[0]}->{pro.k}")
plt.xlabel("iteration #")
plt.ylabel(r"$\Vert\Delta Q \Vert_F$");
# plt.legend();


In [ ]:
d = Naumann24uDataset(Naumann24uDataset.sub_datasets[2])
plt.plot(d.stimulations['sample'], '.')

In [ ]:
d.stimulations

In [ ]:
plt.scatter(d.neuron_df['x'], d.neuron_df['y'])